In [1]:
import math
from typing import Tuple

import numpy as np
import pandas as pd
from scipy.stats import binom

pd.set_option("display.max_columns", None)

In [6]:
snv_file_path = "/home/junkhann/daten/snvs_with_segments/snvs_with_segments_4100314.vcf.gz"

In [7]:
with open("header_names.txt") as file:
    header_names = file.readlines()

header_names = [h.strip("\n") for h in header_names]
len(header_names)

81

In [9]:
with open("snv_usecols.txt") as file:
    usecols_list = file.readlines()

usecols_list = [h.strip("\n") for h in header_names]

usecols_list

['#CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'INFO',
 'FORMAT',
 'sample_control',
 'sample_tumor',
 'DBSNP',
 '1K_GENOMES',
 'PHASE3_1K_GENOMES',
 'ExAC_nonTCGA',
 'LOCAL_CONTROL_2',
 'LOCAL_CONTROL_3',
 'LOCAL_CONTROL_4',
 'ANNOVAR_FUNCTION',
 'GENE',
 'EXONIC_CLASSIFICATION',
 'ANNOVAR_TRANSCRIPTS',
 'SEGDUP',
 'CYTOBAND',
 'MAPABILITY',
 'HISEQDEPTH',
 'SIMPLE_TANDEMREPEATS',
 'REPEAT_MASKER',
 'DUKE_EXCLUDED',
 'DAC_BLACKLIST',
 'SELFCHAIN',
 'CpGislands',
 'CgiMountains',
 'Enhancers',
 'miRNAs_snoRNAs',
 'miRBase18',
 'miRNAtargets',
 'phastConsElem20bp',
 'TFBScons',
 'DNMT3_BS',
 'COSMIC',
 'COSMIC_indels',
 'chromosome',
 'start',
 'end',
 'crest',
 'length',
 'tcnNbrOfLoci',
 'tcnMean',
 'tcnNbrOfSNPs',
 'tcnNbrOfHets',
 'dhNbrOfLoci',
 'map',
 'dhMax',
 'cluster',
 'neighbour',
 'distDH',
 'errorSNP',
 'distTcn',
 'errorLength',
 'totalError',
 'minStart',
 'maxStart',
 'minStop',
 'maxStop',
 'area',
 'peaks',
 'meanCovT',
 'meanCovB',
 'tcnMeanRaw',
 'AF

In [29]:
df_snv = pd.read_csv(snv_file_path, sep='\t', names=header_names, usecols=usecols_list, low_memory=False)
df_snv.shape

(3829904, 81)

In [7]:
df_snv.head()

,#CHROM,POS,REF,ALT,INFO,sample_control,sample_tumor,start,end,genotype,TCN,PID
0,1,842825,A,G,BRF=0.21;FR=1;HP=2;HapScore=1;MGOF=16;MMLQ=16;...,"1/1:-103.9,-9.93,0:16:99:33:33","1/1:-69.7,-6.3,0:12:63:22:22",840009,12839977,1:1,2,4100314
1,1,844300,C,G,BRF=0.8;FR=1;HP=3;HapScore=1;MGOF=46;MMLQ=32;M...,"1/1:-55.7,-4.6,0:46:46:16:16","1/1:-50.5,-4.19,0:41:42:13:13",840009,12839977,1:1,2,4100314
2,1,844324,G,A,BRF=0.65;FR=1;HP=2;HapScore=2;MGOF=60;MMLQ=15;...,"1/1:-30.9,-3.27,0:60:33:14:14","1/1:-20.7,-2.37,0:52:24:11:10",840009,12839977,1:1,2,4100314
3,1,844343,A,G,BRF=0.49;FR=1;HP=1;HapScore=4;MGOF=61;MMLQ=13;...,"1/1:-27.5,-3.03,0:45:30:11:10","1/1:-17.5,-3.58,0:61:36:7:7",840009,12839977,1:1,2,4100314
4,1,844476,T,G,BRF=0.81;FR=0.999;HP=4;HapScore=2;MGOF=13;MMLQ...,"1/1:-3.4,-0,0:9:3:5:4","./.:-3.8,-0,0:13:3:1:1",840009,12839977,1:1,2,4100314


In [8]:
def get_normal_genotype(df: pd.DataFrame) -> str:
    gt = df["sample_control"].split(":")[0]
    if gt == "1/0" or gt == "0/1":
        return "1,1"
    if gt == "0/0":
        return "2,0"
    if gt == "1/1":
        return "0,2"

In [9]:
df_snv["normal_genotype"] = df_snv.apply(get_normal_genotype, axis=1)

In [10]:
df_snv.shape

(3829904, 13)

In [11]:
# get purity for sample (PID)
PID = df_snv.PID.values[0]
print(PID)
df_info = pd.read_csv("/home/junkhann/bioinf-d/Data/mmml/cnv_analysis/Identitymatrix_merged_additionalInfo.tsv", delimiter="\t")
purity = df_info.set_index("PID").loc[str(PID), "purity"]
purity

4100314


0.82

In [12]:
genotypes = df_snv["genotype"].unique()
genotypes


array(['1:1', nan, '2:1', '0:0', '1:0', 'sub:1', 'sub:sub'], dtype=object)

In [13]:
def get_tumor_genotype_with_qualities(df: pd.DataFrame, purity: float) -> tuple[str, float]:
    gt = df["genotype"]
    
    if (gt != gt) or np.isnan(purity):
        genotype = "no_gt"
        quality_score = float('nan')
    elif "sub" in gt:
        genotype = "sub"
        quality_score = float('nan')
    else:
        x = int(gt.split(":")[0])
        y = int(gt.split(":")[1])
        if x == y:
            genotype = f"{x},{y}"
            quality_score = 0
        else:
            try:
                # maximum likelihood with n = NR, k = NV
                n = int(df["sample_tumor"].split(":")[-2])
                k = int(df["sample_tumor"].split(":")[-1])
                tcn_tumor = float(df["TCN"])

                # case 1: x,y (ref, alt)
                vaf_1 = (y * purity) / ((tcn_tumor * purity) + (2 * (1 - purity)))
                prob_1 = binom.pmf(k=k, n=n, p=vaf_1)
                
                # case 2: y,x (ref, alt)
                vaf_2 = (x * purity) / ((tcn_tumor * purity) + (2 * (1 - purity)))
                prob_2 = binom.pmf(k=k, n=n, p=vaf_2)

                # calculating raw phred-scaled likelihoods
                pl_1 = -10 * math.log10(prob_1)
                pl_2 = -10 * math.log10(prob_2)

                # calculating quality score (capped at 99)
                quality_score = min(99, abs(pl_1 - pl_2))

                if prob_1 > prob_2:
                    ref = x
                    alt = y
                else:
                    ref = y
                    alt = x
                
                genotype = f"{ref},{alt}"
                
            except ValueError:
                genotype = "no_gt"
                quality_score = float('nan')
    
    return (genotype, quality_score)
        

In [14]:
df_snv["tumor_genotype"], df_snv["quality_score"] = zip(*df_snv.apply(lambda x: get_tumor_genotype_with_qualities(x, purity), axis=1))
df_snv.shape

(3829904, 15)

In [15]:
def get_total_read_count_tumor(df: pd.DataFrame) -> int:
    trc = df["sample_tumor"].split(":")[-2]
    return trc

In [16]:
def get_total_read_count_normal(df: pd.DataFrame) -> int:
    trc = df["sample_control"].split(":")[-2]
    return trc

In [17]:
df_snv["reads_normal"] = df_snv.apply(get_total_read_count_normal, axis=1)

In [18]:
df_snv["reads_tumor"] = df_snv.apply(get_total_read_count_tumor, axis=1)

In [27]:
df_snv[df_snv["quality_score"] > 0 ]

,#CHROM,POS,REF,ALT,INFO,sample_control,sample_tumor,start,end,genotype,TCN,PID,normal_genotype,tumor_genotype,quality_score,reads_normal,reads_tumor
156820,1,144920112,G,A,BRF=0.18;FR=0.5;HP=2;HapScore=1;MGOF=37;MMLQ=1...,"1/0:-107.83,0,-82.43:37:99:71:39","1/0:-102.9,0,-170.9:30:99:93:35",144920087,145739956,2:1,3,4100314,"1,1","2,1",27.545335,71,93
156821,1,144920155,A,G,BRF=0.13;FR=0.5;HP=2;HapScore=1;MGOF=41;MMLQ=1...,"0/1:-88.14,0,-74.54:41:99:60:34","0/1:-88.03,0,-145.73:33:99:83:32",144920087,145739956,2:1,3,4100314,"1,1","2,1",20.535875,60,83
156822,1,144920210,G,A,BRF=0.1;FR=0.5;HP=1;HapScore=1;MGOF=27;MMLQ=15...,"1/0:-75.68,0,-70.18:27:99:58:29","1/0:-95.52,0,-146.22:23:99:87:37",144920087,145739956,2:1,3,4100314,"1,1","2,1",3.192896,58,87
156823,1,144920308,G,A,BRF=0.14;FR=0.5;HP=2;HapScore=2;MGOF=21;MMLQ=2...,"1/0:-62.26,0,-36.36:21:99:53:24","1/0:-57.68,0,-93.58:16:99:72:25",144920087,145739956,2:1,3,4100314,"1,1","2,1",32.442056,53,72
156824,1,144920425,C,T,BRF=0.21;FR=0.5;HP=4;HapScore=1;MGOF=36;MMLQ=1...,"0/1:-59.24,0,-65.94:36:99:48:22","0/1:-52.07,0,-129.27:28:99:80:25",144920087,145739956,2:1,3,4100314,"1,1","2,1",50.773895,48,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102442,14,106494503,G,A,BRF=0.13;FR=0.25;HP=3;HapScore=1;MGOF=10;MMLQ=...,"1/0:-22.48,0,-47.28:5:99:24:9","0/0:0,-7.83,-84.9:10:78:28:0",106350732,106552286,1:0,1,4100314,"1,1","1,0",99.000000,24,28
3102449,14,106496138,G,A,BRF=0.19;FR=0.25;HP=5;HapScore=1;MGOF=13;MMLQ=...,"1/0:-42.06,0,-69.26:13:99:38:13","0/0:0,-9.03,-104.7:10:90:33:0",106350732,106552286,1:0,1,4100314,"1,1","1,0",99.000000,38,33
3102466,14,106504370,C,T,BRF=0.18;FR=0.25;HP=4;HapScore=1;MGOF=22;MMLQ=...,"0/1:-55.36,0,-62.66:22:99:42:21","0/0:0,-8.43,-98.5:10:84:28:0",106350732,106552286,1:0,1,4100314,"1,1","1,0",99.000000,42,28
3102467,14,106504413,C,A,BRF=0.23;FR=0.25;HP=1;HapScore=1;MGOF=16;MMLQ=...,"1/0:-71.75,0,-71.25:16:99:50:27","0/0:0,-8.13,-92.4:8:81:27:0",106350732,106552286,1:0,1,4100314,"1,1","1,0",99.000000,50,27
